## Use Tensorboard to plot training and validation loss

#### Goal
- Build a 2-convolutional-layered classifier
- Visualize the graph
- Visualize training and validation loss

In [1]:
import tensorflow as tf

## Build the graph

In [2]:
with tf.name_scope('Input_'):
    x = tf.placeholder(tf.float32, shape=[None, 784],name='X0')
    x_image = tf.reshape(x, [-1,28,28,1],name='X_img')
    y = tf.placeholder(tf.float32, shape=[None, 10],name='Labels')

In [3]:
with tf.name_scope('Weight_'):
    W1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1),name='ConvK1')
    W2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1),name='ConvK2')
    W_f = tf.Variable(tf.truncated_normal([7*7*64,1024], stddev=0.1),name='Wf1')
    W_f2 = tf.Variable(tf.truncated_normal([1024,10], stddev=0.1),name='Wf2')


In [4]:
with tf.name_scope('Bias_'):
    b = tf.Variable(tf.constant(0.1, shape=[32]),name='ConvB1')
    b2 = tf.Variable(tf.constant(0.1, shape=[64]),name='ConvB2')
    b_f = tf.Variable(tf.constant(0.1, shape=[1024]),name='Bf1')
    b_f2 = tf.Variable(tf.constant(0.1, shape=[10]),name='Bf2')

In [5]:
conv = tf.nn.conv2d(x_image, W1,strides=[1,1,1,1], padding='SAME',name='ConvFltr1')
conv1 = tf.nn.relu(conv+b,name='ReLU1')
conv_p = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME',name='Pool1')
conv_ = tf.nn.conv2d(conv_p, W2,strides=[1,1,1,1], padding='SAME',name='ConvFltr2')
conv2 = tf.nn.relu(conv_+b2,name='ReLU2')
conv_p2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME',name='Pool2')

In [6]:
h_p = tf.reshape(conv_p2,[-1,7*7*64],name='Flattend')
h_f = tf.nn.relu(tf.add(tf.matmul(h_p,W_f),b_f),name='ReLU3')

In [7]:
yhat = tf.add(tf.matmul(h_f,W_f2),b_f2,name='Logits')
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat),name='Loss')

In [8]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#### Add the summary for tensorboard

In [9]:
loss_summary = tf.summary.scalar('XEntropy', cross_entropy)
#merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('tmp/train', tf.get_default_graph())
validation_writer = tf.summary.FileWriter('tmp/validation')

### Read in data

In [10]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
''' local MNIST data repos:
C:\\gitLocal\\ML\\data\\MNIST_data
C:\\Users\\daij12\\MNIST_data
'''
mnist_dir = 'C:\\gitLocal\\ML\\tensorflow\\MNIST_data'
mnist_data = read_data_sets(train_dir=mnist_dir,one_hot=True)

Extracting C:\gitLocal\ML\tensorflow\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\gitLocal\ML\tensorflow\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\gitLocal\ML\tensorflow\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\gitLocal\ML\tensorflow\MNIST_data\t10k-labels-idx1-ubyte.gz


### Run the graph

In [11]:
sess = tf.InteractiveSession()

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
for b in range(1000):
    btch_x, btch_y = mnist_data.train.next_batch(50)
    train_step.run(feed_dict={x:btch_x,y:btch_y})
    if b % 5==0:
        _,summary = sess.run([cross_entropy,loss_summary],feed_dict={x:btch_x,y:btch_y})
        train_writer.add_summary(summary, b)
        btch_x, btch_y = mnist_data.validation.next_batch(50)
        _,summary = sess.run([cross_entropy,loss_summary],feed_dict={x:btch_x,y:btch_y})
        validation_writer.add_summary(summary, b)

#### The visual in tensorboard

![training_error and validation_error](./graph-X2.PNG)